In [37]:
import numpy as np
from pandas import DataFrame
arr = DataFrame(np.arange(12).reshape((3,4)), columns=['al', 'beta', 'gamma', 'lambda'])

import glob
image_filenames = glob.glob('./imagenet-dogs/n02*/*.jpg')
from itertools import groupby
from collections import defaultdict
image_filenames[:2]

image_filename_with_breed = map(lambda filename: (filename.split("/")[2], filename), image_filenames)
image_filename_with_breed[:2]

[('n02085620-Chihuahua',
  './imagenet-dogs/n02085620-Chihuahua/n02085620_10074.jpg'),
 ('n02085620-Chihuahua',
  './imagenet-dogs/n02085620-Chihuahua/n02085620_10131.jpg')]

In [38]:
training_dataset = defaultdict(list)
testing_dataset = defaultdict(list)
for dog_breed, breed_images in groupby(image_filename_with_breed, lambda x: x[0]):
    for i, breed_image in enumerate(breed_images):
        if i % 5 == 0:
            testing_dataset[dog_breed].append(breed_image[1])
        else:
            training_dataset[dog_breed].append(breed_image[1])
    
    # Check that each breed includes at least 18% of the images for testing
    breed_training_count = float(len(training_dataset[dog_breed]))
    breed_testing_count = float(len(testing_dataset[dog_breed]))

#     print('dog breed:%s - train count:%d, test count:%d, ratio:%f' % 
#           (dog_breed, breed_training_count, breed_testing_count, 
#            breed_testing_count/(breed_training_count+breed_testing_count)))

    assert round(breed_testing_count / (breed_training_count + breed_testing_count), 2) > 0.18, \
        'Not enoughh testing images.'


In [39]:
import tensorflow as tf
def write_records_file(dataset, record_location):
    """
    Fill a TFRecords file with the images found in 'dataset' and include their category.
    Parameters
    ----------
    dataset: dict(list)
        Dictionary with each key being a label for the list of image filenames of its value.
    record_location: str
        Location to store the TFRecord output.
    """
    writer = None
    
    sess = tf.Session()
    # Enumerating the dataset because the current index is used to breakup the files if they get over 100
    # images to avoid a slowdown in writing.
    current_index = 0
    for breed, images_filenames in dataset.items():
        for image_filename in images_filenames:
            if current_index < 13800:
                current_index += 1
                continue
            elif current_index >= 13800 and current_index < 13900:
                print("{0} - file name:{1}".format(current_index, image_filename))
            
            if current_index % 100 == 0:
                print("START {0} - file name:{1}".format(current_index, image_filename))
                if writer:
                    writer.close()
                record_filename = '{record_location}-{current_index}.tfrecords'.format(
                    record_location=record_location, current_index=current_index)
                writer = tf.python_io.TFRecordWriter(record_filename)
            current_index += 1
        
            image_file = tf.read_file(image_filename)
            # In ImageNet dogs, there are a few images which TensorFlow doesn't recognize as JPEGs. This
            # try/catch will ignore those images
            try:
                image = tf.image.decode_jpeg(image_file)
            except:
                print(image_filename)
                continue
            
            # Converting to grayscale saves processing and memory but isn't required
            grayscale_image = tf.image.rgb_to_grayscale(image)
            resized_image = tf.image.resize_images(grayscale_image, (250, 151))
            
            # tf.cast is used here because the resized images are floats but haven't been converted into
            # image floats where an RGB value is between [0,1).
            image_bytes = sess.run(tf.cast(resized_image, tf.uint8)).tobytes()
            
            # Instead of using the label as a string, it'd be more efficient to turn it into either an
            # integer index or a one-hot encoded rank one tensor.
            # https://en.wikipedia.org/wiki/One-hot
            image_label = breed.encode('utf-8')
            
            example = tf.train.Example(features=tf.train.Features(feature={
                        'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_label])),
                        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
                    }))
            writer.write(example.SerializeToString())
            
    if writer:
        writer.close()
    sess.close()
    print('end write_records_file')

write_records_file(testing_dataset, "./output/testing-images/testing-image")
write_records_file(training_dataset, "./output/training-images/training-image")

write_records_file(testing_dataset, './output-testing-images-testing-image')
write_records_file(training_dataset, './output-training-images-trainging-image')
            

13800 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_13482.jpg
START 13800 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_13482.jpg
13801 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_13586.jpg
13802 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_13611.jpg
13803 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_13955.jpg
13804 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_14052.jpg
13805 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_14081.jpg
13806 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_14126.jpg
13807 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_14297.jpg
13808 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_14509.jpg
13809 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_14594.jpg
13810 - file name:./imagenet-dogs/n02105855-Shetland_sheepdog/n02105855_14653.jpg
13811 - fi

In [45]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once('~/pooh/poohdrop/large-data/tf_inteligence/output/training-images/*.tfrecords'))

In [46]:
reader = tf.TFRecordReader()
_, serialized = reader.read(filename_queue)


In [47]:
features = tf.parse_single_example(
    serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'image': tf.FixedLenFeature([], tf.string),
    })
record_image = tf.decode_raw(features['image'], tf.uint8)

In [48]:
record_image

<tf.Tensor 'DecodeRaw_1:0' shape=(?,) dtype=uint8>

In [49]:
image = tf.reshape(record_image, [250, 151, 1])
label = tf.cast(features['label'], tf.string)
min_after_dequeue = 10
batch_size = 3
capacity = min_after_dequeue + 3 * batch_size
image_batch, label_batch = tf.train.shuffle_batch(
    [image, label], batch_size=batch_size, capacity=capacity,
    min_after_dequeue=min_after_dequeue)

In [58]:
# Converting the images to a float of [0,1) to match the expected input to convolution2d
float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)
conv2d_layer_one = tf.contrib.layers.convolution2d(
    float_image_batch,
    num_outputs=32,
    kernel_size=(5,5),
    activation_fn=tf.nn.relu,
    weights_initializer=tf.random_normal_initializer(),
    stride=(2,2),
    trainable=True)
pool_layer_one = tf.nn.max_pool(conv2d_layer_one,
                                ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding='SAME')

In [59]:
# Note, the first and last dimension of the convolution output hasn't changed but the
# middle two dimensions have
conv2d_layer_one.get_shape(), pool_layer_one.get_shape()

(TensorShape([Dimension(3), Dimension(125), Dimension(76), Dimension(32)]),
 TensorShape([Dimension(3), Dimension(63), Dimension(38), Dimension(32)]))